In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import pandas as pd
import scipy.spatial.distance as sdst
from skimage import io as skio
#
sys.path.append('../../..')
from biodl.deephd.dhd_data import load_one_sample, _load_idx
from biodl.bio_utils import parallel_tasks_run_def

In [2]:
path_idx = '/mnt/data1T/data/annaha/homod/idxok.txt'
data_idx = _load_idx(path_idx)


In [3]:
def get_dst_mat(sample: dict) -> np.ndarray:
    ret = []
    for atom_type in ('ca', 'cb'):
        x1, x2 = sample[f'coords_{atom_type}']
        dst_x1x1 = sdst.cdist(x1, x1).astype(np.float32)
        dst_x1x2 = sdst.cdist(x1, x2).astype(np.float32)
        ret.append(dst_x1x1)
        ret.append(dst_x1x2)
    ret = np.concatenate(ret, axis=1)
    return ret

def prepare_preview_img(row: pd.Series) -> bool:
    path_pdb = row['path_pdb_abs']
    path_img = os.path.splitext(path_pdb)[0] + '_preview.png'
    if os.path.isfile(path_img):
        return True
    data_sample = load_one_sample(row)
    dst_img = get_dst_mat(data_sample)
    dst_img = (255. * dst_img / dst_img.max()).astype(np.uint8)
    skio.imsave(path_img, dst_img)
    return True

In [4]:
tasks_data = [{'row': x} for _, x in data_idx.iterrows() ]

# for irow, row in data_idx.iterrows():
#     data_sample = load_one_sample(row)    
#     dst_img = get_dst_mat(data_sample)
#     dst_img = (255. * dst_img / dst_img.max()).astype(np.uint8)
#     break

In [5]:
ret = parallel_tasks_run_def(prepare_preview_img, tasks_data, num_workers=8)

INFO:root::: start (unnamed-task) #data/#threads = 17555/8, use-process(True)
INFO:root:	[0/17555] (l1) (unnamed-task)
INFO:root:	[1756/17555] (l1) (unnamed-task)
INFO:root:	[3512/17555] (l1) (unnamed-task)
INFO:root:	[5268/17555] (l1) (unnamed-task)
INFO:root:	[7024/17555] (l1) (unnamed-task)
INFO:root:	[8780/17555] (l1) (unnamed-task)
INFO:root:	[10536/17555] (l1) (unnamed-task)
INFO:root:	[12292/17555] (l1) (unnamed-task)
INFO:root:	[14048/17555] (l1) (unnamed-task)
INFO:root:	[15804/17555] (l1) (unnamed-task)
INFO:root:		... done, dt ~ 172.74 (s), speed=101.63 (samples/s) [unnamed-task]
